# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
clean_city_data = pd.read_csv("../WeatherPy/cleancitydata.csv")
clean_city_data


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Salinas,36.6777,-121.6555,54.01,81,1,3.44,US,1634999287
1,Necochea,-38.5473,-58.7368,61.65,56,71,10.00,AR,1634999287
2,Rikitea,-23.1203,-134.9692,74.41,81,41,20.94,PF,1634998954
3,Ushuaia,-54.8000,-68.3000,42.46,56,40,12.66,AR,1634998899
4,Chui,-33.6971,-53.4616,65.43,57,100,3.58,UY,1634999072
...,...,...,...,...,...,...,...,...,...
560,Avera,33.1940,-82.5271,62.47,65,0,6.67,US,1634998975
561,Niimi,34.9833,133.4667,51.15,83,8,2.93,JP,1634999416
562,Mariestad,58.7097,13.8237,49.37,63,13,2.28,SE,1634999417
563,Quang Ngai,15.1167,108.8000,73.13,96,100,4.79,VN,1634999137


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = clean_city_data[["Lat", "Lng"]]

# Fill NaN values and convert to float
rating = clean_city_data["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
#Narrow down the DataFrame to find your ideal weather condition.
Temp_data_high = clean_city_data[(clean_city_data["Max Temp"] <80)]
Temp_data_high

Temp_data_low = Temp_data_high[(Temp_data_high["Max Temp"] >72)]
Temp_data_low

Wind_Speed_data = Temp_data_low[(Temp_data_low["Wind Speed"] <7)]
Wind_Speed_data

my_cities_df = Wind_Speed_data[(Wind_Speed_data["Cloudiness"] ==0)]
my_cities_df 

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
139,Goshāinganj,26.5833,82.3833,75.83,60,0,3.42,IN,1634999311
154,Rewa,24.5333,81.3000,77.50,45,0,1.19,IN,1634999314
248,China,25.7000,-99.2333,74.89,82,0,6.67,MX,1634999334
251,Bokaro Steel City,23.7833,85.9667,73.65,64,0,5.68,IN,1634999334
277,Kumluca,36.3703,30.2869,73.15,57,0,6.24,TR,1634999339
403,Bhabhua,25.0500,83.6167,74.14,63,0,2.46,IN,1634999370
461,Alghero,40.5589,8.3181,73.49,60,0,6.91,IT,1634999384
480,Cascais,38.6979,-9.4215,76.35,51,0,5.99,PT,1634999388
534,Camas,37.4020,-6.0331,78.42,44,0,3.00,ES,1634999404
555,Mangaldai,26.4333,92.0333,75.11,78,0,1.03,IN,1634999414


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [38]:
#Store into variable named hotel_df.
hotel_df = my_cities_df.copy()

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"]=""
hotel_df

# find the closest restaurant of each type to coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    #Set parameters to search for hotels within 5000 meters
    City = f"{hotel_df.loc[index, 'Lat']}, {hotel_df.loc[index, 'Lng']}"
    Vicinity = 5000
    Hotels = "lodging"

    params = {
        "location": City,
        "radius": Vicinity,
        "type": Hotels,
        "key": g_key}

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {Hotel_info}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Retrieving Records for {index}: Closest Hotel is {results[0]['name']}")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 139: Mangaldai.
Retrieving Records for 139: Closest Hotel is suraj Trunk House
------------
Retrieving Results for Index 154: Mangaldai.
Retrieving Records for 154: Closest Hotel is OYO 71149 Jeet Residency
------------
Retrieving Results for Index 248: Mangaldai.
Retrieving Records for 248: Closest Hotel is González Reyes Artemio
------------
Retrieving Results for Index 251: Mangaldai.
Retrieving Records for 251: Closest Hotel is Bineshwar Hotel
------------
Retrieving Results for Index 277: Mangaldai.
Retrieving Records for 277: Closest Hotel is TurkuazKöy
------------
Retrieving Results for Index 403: Mangaldai.
Retrieving Records for 403: Closest Hotel is Kohinoor Hotel
------------
Retrieving Results for Index 461: Mangaldai.
Retrieving Records for 461: Closest Hotel is Villa Las Tronas Hotel & Spa
------------
Retrieving Results for Index 480: Mangaldai.
Retrieving Records for 480: Closest Hotel is Hotel Vila Galé Cascais
------------
Retrieving Resu

In [39]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
139,Goshāinganj,26.5833,82.3833,75.83,60,0,3.42,IN,1634999311,suraj Trunk House
154,Rewa,24.5333,81.3000,77.50,45,0,1.19,IN,1634999314,OYO 71149 Jeet Residency
248,China,25.7000,-99.2333,74.89,82,0,6.67,MX,1634999334,González Reyes Artemio
251,Bokaro Steel City,23.7833,85.9667,73.65,64,0,5.68,IN,1634999334,Bineshwar Hotel
277,Kumluca,36.3703,30.2869,73.15,57,0,6.24,TR,1634999339,TurkuazKöy
403,Bhabhua,25.0500,83.6167,74.14,63,0,2.46,IN,1634999370,Kohinoor Hotel
461,Alghero,40.5589,8.3181,73.49,60,0,6.91,IT,1634999384,Villa Las Tronas Hotel & Spa
480,Cascais,38.6979,-9.4215,76.35,51,0,5.99,PT,1634999388,Hotel Vila Galé Cascais
534,Camas,37.4020,-6.0331,78.42,44,0,3.00,ES,1634999404,Hotel Derby
555,Mangaldai,26.4333,92.0333,75.11,78,0,1.03,IN,1634999414,Hotel Siddarth Palace


In [40]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [42]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))